In [1]:
import numpy as np
import os
import threading
import operator
import pickle
import re
import pandas as pd


start_time = 1190146243
end_time = 1192994591

def load_data(input_name):
    with open(input_name, 'rb') as f:
        data = pickle.load(f)
    return data
def dump_data(data, output_name):
    with open(output_name, 'wb') as f:
        pickle.dump(data, f)




import mwclient
from itertools import islice
import calendar
from urllib.parse import unquote
from urllib.parse import quote
import time

site = mwclient.Site('en.wikipedia.org')

def get_page(title):
    page = site.pages[unquote(unquote(title))]
    return page

# Get revision records

In [2]:
def get_revisions(title, prop='ids|timestamp|flags|comment|user|size'):
    page = site.pages[unquote(unquote(title))]
    revision = page.revisions(start=1192994591, end=1190146243, prop = prop)
    return revision

def revision_dataframe(title):
    revisions = get_revisions(title, prop='ids|timestamp|user|size')
    start = time.time()
    series = []
    for rev in revisions:
        rev['timestamp'] = calendar.timegm(rev['timestamp'])
        if 'user' not in rev:
            rev['user'] = ''
        if 'parentid' not in rev:
            rev['parentid'] = ''
        
                
        record = pd.Series(rev)
        series.append(record)
    if series:
        return pd.DataFrame(series, index = range(len(series)))
    else:
        return None
        
def collect_revisions(titles, nworkers = 4):
    results = []
    keys = []
    i = 0
    threads = [None] * nworkers
    flags = [True] * nworkers
    
    def target(index, flags, results, keys,title):
        try:
            df = revision_dataframe(title)
            f.write('word {} succeed\n'.format(title))
        except:
            f.write('word {} fail\n'.format(title))
            df = None
        if df is not None:
            results.append(df)
            keys.append(title)
            assert(len(results) == len(keys))
        flags[index] = True
    f = open('log_spider.txt','w')
    for title in titles:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        t = threading.Thread(target = target, args = [index, flags, results, keys,title])
        threads[index] = t
        t.start()
        i += 1
        f.write('Processing word {}\n'.format(i))
        f.flush()
    for thread in threads:
        if thread:
            thread.join()
    return pd.concat(results, keys=keys)

In [14]:
#word_series = load_data("data/dump/hot_words_100000.pkl")
#words_list = word_series.index
#revision_df = collect_revisions(words_list)
#dump_data(revision_df,"data/dump/revision_records.pkl")

In [17]:
revision_df.to_csv("data/test.csv")

# Get metadata

language

media_num

categories

links

In [6]:
from ipywidgets import IntProgress

def get_metadata(title):
    meta = {}
    page = site.pages[unquote(unquote(title))]
    categories = list(page.categories(generator=False))
    langs = list(page.langlinks())
    links = list(page.links(generator=False))
    nimages = len(list(page.images(generator=False)))
    meta['languages'] = langs
    meta['links'] = links
    meta['categories'] = categories
    meta['image_num'] = nimages
    return meta

def collect_metas(titles, nworkers = 4):
    metas = {}
    i = 0
    threads = [None] * nworkers
    flags = [True] * nworkers
    
    def target(index, flags, title):
        try:
            meta = get_metadata(title)
        except:
            meta = None
        if meta is not None:
            metas[title] = meta
        flags[index] = True
    p =  IntProgress(max = 10000)  
    p.discription = "Running"
    display(p)
    for title in titles:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        t = threading.Thread(target = target, args = [index, flags, title])
        threads[index] = t
        t.start()
        i += 1
        p.value += 1
        
    for thread in threads:
        if thread:
            thread.join()
    return metas

In [ ]:
word_series = load_data("data/dump/hot_words_100000.pkl")
words_list = word_series.index
delta = 10000
for j in range(1, 10):
    if os.path.isfile("data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1))):
        print("data/dump/metadata_{}-{}.pkl exists".format(delta * j, delta * (j+1)))
        continue
    print("data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1)))
    metas = collect_metas(words_list[delta * j: delta * (j+1)])
    dump_data(metas, "data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1)))

data/dump/metadata_10000-20000.pkl exists
data/dump/metadata_20000-30000.pkl exists
data/dump/metadata_30000-40000.pkl exists
data/dump/metadata_40000-50000.pkl exists
data/dump/metadata_50000-60000.pkl exists
data/dump/metadata_60000-70000.pkl exists
data/dump/metadata_70000-80000.pkl


IntProgress(value=0, max=10000)

data/dump/metadata_80000-90000.pkl


IntProgress(value=0, max=10000)

data/dump/metadata_90000-100000.pkl


IntProgress(value=0, max=10000)

99999